## Multi-head attention transformer
### Encoder and Decoder
### (With masking)

Pytorch's implementation (in built)

NOTE :- A new exmple must be used for testing the masked attention

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size,max_seq_len, d_model=4, nhead=2, num_encoder_layers=1, num_decoder_layers=1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0, max_len=max_seq_len)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        
        return output

    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 6  # Dimension of the model
num_heads = 2
max_seq_len = 4
model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model=d_model, max_seq_len = max_seq_len, nhead=num_heads)

# Source sentence in the source language
# Source token indexes from src vocabulary
src_sentence = torch.tensor([[0], [1], [2], [3]]) 



# Target sentence in the target language (translation of the source sentence)
# Target token indexes from tgt vocabulary
tgt_sentence = torch.tensor([[1], [0], [3], [3]])

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output)



Source embeddings :- 

tensor([[[-0.1344,  0.5592, -1.1815,  0.4653, -0.7154, -1.0503]],

        [[ 0.0315,  0.4340, -0.7013,  2.0887, -0.2541,  1.3491]],

        [[-0.8416, -0.0752, -0.0664, -0.7628,  0.1541,  1.2462]],

        [[-1.3622, -0.6211, -0.8772,  0.4195, -0.0854,  0.3847]]],
       grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[ 0.5611,  0.5838,  1.1292,  0.8900,  1.5531, -1.4184]],

        [[-0.5187, -0.1657, -0.2895, -0.3737, -0.3584,  1.3122]],

        [[ 0.0646,  1.1269, -0.5113,  0.5007,  1.5989,  0.3129]],

        [[ 0.0646,  1.1269, -0.5113,  0.5007,  1.5989,  0.3129]]],
       grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-0.1344,  1.5592, -1.1815,  1.4653, -0.7154, -0.0503]],

        [[ 0.0315,  1.4340, -0.7013,  3.0887, -0.2541,  2.3491]],

        [[-0.8416,  0.9248, -0.0664,  0.2372,  0.1541,  2.2462]],

        [[-1.3622,  0.3789, -0.8772,  1.4195, -0.0854,  1.3847]]],
       grad_fn=<AddBackward0>)

Posi

## Functions to get the manual calulations for each component

In [34]:
from torch import functional as F

# Using the state dictionary to get the intermediate outputs
state_dict = model.state_dict()

need_weights = False

src_mask = None

tgt_mask = None

memory_mask = None

embed_dim = 6

num_heads = 2

max_seq_len = 4


In [35]:
def look_up_table(sentence, vocab_embeds, embedding):

    for i in range(sentence.size(0)):
        for j in range(sentence.size(1)):
            # Get the index for the current word token index in the sequence
            word_index = sentence[i, j].item()

            if word_index < 0 or word_index >= vocab_embeds.size(0):
                raise ValueError(f"Invalid word index: {word_index}")

            # Lookup the corresponding embedding vector for the word
            embedding[i, j, :] = vocab_embeds[word_index, :]

            print(f"Word index: {word_index}, Embedding: {vocab_embeds[word_index, :]}")
    print()
    

    return embedding

### Embeddings and Positional encoding

In [36]:
def get_embedding_outputs(src_sentence,  tgt_sentence, max_seq_len, state_dict, d_model):

    src_vocab_embeds = state_dict["src_embedding.weight"]

    src_embedding = torch.zeros(src_sentence.size(0), src_sentence.size(1), d_model)
    print("Source sentence embedding")
    src_embedding =  look_up_table(src_sentence, src_vocab_embeds, src_embedding)
    print(src_embedding.shape)

    tgt_vocab_embeds = state_dict["tgt_embedding.weight"]

    tgt_embedding = torch.zeros(tgt_sentence.size(0), tgt_sentence.size(1), d_model)


    print("Target sentence embedding")
    tgt_embedding =  look_up_table(tgt_sentence, tgt_vocab_embeds, tgt_embedding)

    pe = PositionalEncoding(d_model = d_model, dropout=0, max_len=max_seq_len)

    print("PE of src :")
    print(pe(src_embedding))
    print()
    print("PE of tgt :")
    print(pe(tgt_embedding))
    print()

    pe_src_embeds = src_embedding + pe(src_embedding)

    pe_tgt_embeds = tgt_embedding + pe(tgt_embedding)

    print("PE source embeddings : \n")
    print(pe_src_embeds)
    print()

    print("PE target embeddings : \n")
    print(pe_tgt_embeds)
    print()

    return pe_src_embeds, pe_tgt_embeds



In [37]:
pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence=src_sentence,  tgt_sentence=tgt_sentence, state_dict=state_dict, max_seq_len=max_seq_len, d_model = d_model)


Source sentence embedding
Word index: 0, Embedding: tensor([-0.1344,  0.5592, -1.1815,  0.4653, -0.7154, -1.0503])
Word index: 1, Embedding: tensor([ 0.0315,  0.4340, -0.7013,  2.0887, -0.2541,  1.3491])
Word index: 2, Embedding: tensor([-0.8416, -0.0752, -0.0664, -0.7628,  0.1541,  1.2462])
Word index: 3, Embedding: tensor([-1.3622, -0.6211, -0.8772,  0.4195, -0.0854,  0.3847])

torch.Size([4, 1, 6])
Target sentence embedding
Word index: 1, Embedding: tensor([ 0.5611,  0.5838,  1.1292,  0.8900,  1.5531, -1.4184])
Word index: 0, Embedding: tensor([-0.5187, -0.1657, -0.2895, -0.3737, -0.3584,  1.3122])
Word index: 3, Embedding: tensor([ 0.0646,  1.1269, -0.5113,  0.5007,  1.5989,  0.3129])
Word index: 3, Embedding: tensor([ 0.0646,  1.1269, -0.5113,  0.5007,  1.5989,  0.3129])

PE of src :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE of tgt :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE source embeddings : 

tensor([[[-0.1344,  1.5592, -1.1815,  1.4653, -0.7154, -0.0503]],

        [[ 0.0315,  

## Encoder function to display the intermediate outputs and get the final outputs from the encoder

### Self attention 

In [88]:
def atten_product_needs_wts_false(Q, V, K, bsz, head_dim, src_len, tgt_len, attn_mask):


    # *** For multi-head attention ***
    #  (bsz*num_heads, src_len , head_dim) -> (bsz, num_heads, tgt_len, head_dim)
    Q1 = Q.view(bsz, num_heads, tgt_len, head_dim)
    K1 = K.view(bsz, num_heads, src_len, head_dim)
    V1 = V.view(bsz, num_heads, src_len, head_dim)

    L, S = Q1.size(-2), K1.size(-2)

    scale_factor = 1 / math.sqrt(Q1.size(-1)) 
    # scale_factor = 1
    attn_bias = torch.zeros(L, S, dtype=Q1.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_mask.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask

    # (bsz, num_heads, tgt_len, head_dim) @ (bsz, num_heads, head_dim, tgt_len) -> (bsz, num_heads, tgt_len, tgt_len) 
    attn_weight = Q1 @ K1.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias

    # (bsz, num_heads, tgt_len, tgt_len) 
    attn_weight = torch.softmax(attn_weight, dim=-1)

    print("Attention weights = ")
    print(attn_weight.shape, attn_weight)

    # (bsz, num_heads, tgt_len, tgt_len) @ (bsz, num_heads, tgt_len, head_dim) -> (bsz, num_heads, tgt_len, head_dim) 
    attn_output = attn_weight @ V1

    # (bsz*tgt_len, embed_dim)
    attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(bsz * tgt_len, embed_dim)

    return attn_output



def atten_product_needs_wts_true(Q, K, V, bsz, tgt_len, attn_mask):

    # *** For multi-head attention ***
    #  (bsz*num_heads, src_len , head_dim)
    B, Nt, E = Q.shape

    Q_scaled = Q / math.sqrt(E)

    if attn_mask is not None:
        temp_pdt_matrix = torch.baddbmm(attn_mask, Q_scaled, K.transpose(-2, -1))
    else:
        temp_pdt_matrix = torch.bmm(Q_scaled, K.transpose(-2, -1))

    attn_wt_matrix = torch.nn.functional.softmax(temp_pdt_matrix, dim=-1)

    attn_output = torch.bmm(attn_wt_matrix, V)

    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)

    print("Encoder Attention output = ")
    print(attn_output)
    print()

    return attn_output, attn_wt_matrix


In [71]:
def get_qkv(query, key, value ,W, b):

    # embed_dim
    E = query.size(-1)

    if key is value:
        if query is key:
            
            # (src_len, bsz, embed_dim) @ (embed_dim*num_heads, embed_dim).T -> (src_len, bsz, embed_dim*num_heads)
            tempop1 = query@W.T

            # reshape to 3, E and not E, 3 is deliberate for better memory coalescing and keeping same order as chunk()
            tempop1 = tempop1.unflatten(-1, (3, E)).unsqueeze(0).transpose(0, -2).squeeze(-2).contiguous()

            # (src_len, bsz, embed_dim)
            return tempop1[0], tempop1[1], tempop1[2]
        

        else:

            # (embed_dim*1, embed_dim)
            # (embed_dim*2, embed_dim)
            W_q, W_kv = W.split([E, E * 2])

            if b is None:
                b_q = b_kv = None
            else:
                b_q, b_kv = b.split([E, E * 2])

            # (src_len, bsz, embed_dim) @ (embed_dim*1, embed_dim).T -> (src_len, bsz, embed_dim)
            q_matmul = query@W_q.T

            # (src_len, bsz, embed_dim) @ (embed_dim*2, embed_dim).T -> (src_len, bsz, embed_dim*2)
            kv_matmul = key@W_kv.T

            kv_matmul = kv_matmul.unflatten(-1, (2, E)).unsqueeze(0).transpose(0, -2).squeeze(-2).contiguous()

            # (src_len, bsz, embed_dim)
            return q_matmul, kv_matmul[0], kv_matmul[1]

    else:

        W_q, W_k, W_v = W.chunk(3)
        if b is None:
            b_q = b_k = b_v = None
        else:
            b_q, b_k, b_v = b.chunk(3)


        q_matmul = query@W_q.T
        k_matmul = key@W_k.T
        v_matmul = value@W_v.T

        return q_matmul, k_matmul, v_matmul



    

In [75]:
def encoder_block_attn_output(x, state_dict, layer_num, embed_dim, num_heads, need_weights = False, src_mask = None):

    # (src_len, bsz, embed_dim)
    query_enc = key_enc = value_enc = x

    tgt_len, bsz, embed_dim = x.shape

    # print(x.shape)
    
    # (embed_dim*num_heads, embed_dim)
    W_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads
    
    # (src_len, bsz, embed_dim)
    Q_enc,K_enc,V_enc = get_qkv(query_enc, key_enc, value_enc ,W_enc, b_enc)

    # (1, src_len, bsz, embed_dim)
    Q_enc = Q_enc.unsqueeze(0)
    K_enc = K_enc.unsqueeze(0)
    V_enc = V_enc.unsqueeze(0)

    # (1, src_len, bsz, embed_dim) -> ( bsz*num_heads, src_len , head_dim)
    Q_enc = Q_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_enc = K_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_enc = V_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_enc_{} = ".format(layer_num))
    print(Q_enc)
    print()

    print("K_enc_{} = ".format(layer_num))
    print(K_enc)
    print()

    print("V_enc_{} = ".format(layer_num))
    print(V_enc)
    print()


    src_len = K_enc.size(1)

    attn_mask = src_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    if need_weights is False:

        attn_output = atten_product_needs_wts_false(Q_enc, V_enc, K_enc, bsz, head_dim, src_len, tgt_len, attn_mask)

        return attn_output, src_len, head_dim, None
    
    else:

        attn_enc_output,attn_wt_matrix_enc = atten_product_needs_wts_true(Q_enc, K_enc, V_enc, bsz, tgt_len, attn_mask)

        return attn_enc_output, src_len, head_dim, attn_wt_matrix_enc

    

In [76]:
# need_weights = True

attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(pe_src_embeds, state_dict, layer_num = 0, need_weights = need_weights, embed_dim=embed_dim, num_heads=num_heads)





torch.Size([4, 1, 6])
torch.Size([18, 6])
Q_enc_0 = 
torch.Size([2, 4, 3])
tensor([[[-0.7205,  0.8394, -1.2265],
         [ 0.0944,  1.7884, -0.8154],
         [ 0.3803,  1.0893,  0.3105],
         [ 0.2034,  1.1417, -0.0092]],

        [[-0.1023,  0.5319,  0.1854],
         [-0.0114,  0.0831,  0.6531],
         [-0.4294,  0.6794,  1.1543],
         [ 0.2220,  0.7552,  0.1356]]])

K_enc_0 = 
tensor([[[-0.6167, -0.2793,  0.0149],
         [ 1.3568, -0.3901, -0.2702],
         [ 0.6660, -0.4651,  0.7754],
         [ 0.9486, -0.1334,  0.4902]],

        [[ 0.3259,  1.1248,  0.0105],
         [ 1.6123,  1.7022,  0.0453],
         [ 1.1180,  1.7583, -0.8162],
         [ 0.9340,  1.5642,  0.3434]]])

V_enc_0 = 
tensor([[[ 0.1706, -0.3731, -0.7106],
         [ 0.9077, -1.3990, -0.6533],
         [ 0.5089, -0.4129,  0.0070],
         [ 0.3045, -0.4677, -0.2549]],

        [[-1.5414, -1.2804,  0.9048],
         [-1.3778, -2.2945,  2.0395],
         [ 0.0209, -1.3055,  0.5223],
         [-0.8

In [80]:
state_dict["transformer.encoder.layers.{}.norm1.weight".format(0)].shape

torch.Size([6])

### Post self attention in encoder

In [81]:
def encoder_block_post_attn_output(x, attn_enc_output, state_dict, layer_num, bsz, tgt_len):

    # (bsz*src_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*src_len , embed_dim)
    op_enc_1 = torch.matmul(attn_enc_output, state_dict["transformer.encoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]



    # (bsz*src_len , embed_dim) -> (src_len, bsz, embed_dim)
    attn_enc_output = op_enc_1.view(tgt_len, bsz, attn_enc_output.size(1))


    # Here src is the original passed inputs to the 1st transformer encoder layer which 
    # are pe_src_embeds 

    # (src_len, bsz, embed_dim)
    output_enc_1 = attn_enc_output + x

    #  (src_len, bsz, embed_dim) @ (embed_dim) -> (src_len, bsz, embed_dim) 
    linear_result_enc_1 = output_enc_1*state_dict["transformer.encoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm1.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_1 = torch.nn.LayerNorm(normalized_shape=linear_result_enc_1.shape[2:])
    linear_op_enc_1 = layernorm_enc_1(linear_result_enc_1)


    # Manual Layer Normalization
    x = linear_result_enc_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_1.weight
    b = layernorm_enc_1.bias

    linear_result_enc_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_1f.mean(dim=-1, keepdim=True)
    std = linear_result_enc_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_enc_1 = (linear_result_enc_1f - mean) / (std + epsilon) * w + b


    op_enc_1 = torch.matmul(normalized_result_enc_1, state_dict["transformer.encoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear1.bias".format(layer_num)]
    op_enc_1_relu = torch.nn.functional.relu(op_enc_1)
    op_enc_2 = torch.matmul(op_enc_1_relu, state_dict["transformer.encoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear2.bias".format(layer_num)]


    output_enc_2 = op_enc_2 + linear_op_enc_1
    output_enc_2_norm = output_enc_2*state_dict["transformer.encoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_final = torch.nn.LayerNorm(normalized_shape=output_enc_2_norm.shape[2:])
    output_enc_final = layernorm_enc_final(output_enc_2_norm)


    # Manual Layer Normalization 
    x = output_enc_2_norm

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_final.weight
    b = layernorm_enc_final.bias

    linear_result_enc_2 = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_2.mean(dim=-1, keepdim=True)
    std = linear_result_enc_2.std(dim=-1, unbiased=False, keepdim=True)
    output_enc_final = (linear_result_enc_2 - mean) / (std + epsilon) * w + b

    print("Final Encoder {} Output :".format(layer_num))
    print("norm2(norm1(x + self_atten(x)) + feed_fwd_op)\n")
    print(output_enc_final)
    print()

    # (src_len, bsz, embed_dim) 
    return output_enc_final


In [82]:
tgt_len, bsz, embed_dim = pe_src_embeds.shape
output_enc_final = encoder_block_post_attn_output(pe_src_embeds, attn_enc_output, state_dict, layer_num = 0 , bsz = bsz, tgt_len = tgt_len)

Final Encoder 0 Output :
norm2(norm1(x + self_atten(x)) + feed_fwd_op)

tensor([[[ 0.2594,  1.5738, -0.2733, -0.2970, -1.7484,  0.4855]],

        [[-0.3960,  0.5484, -0.4211,  0.3254, -1.6497,  1.5930]],

        [[-0.5191,  0.6015,  0.3674, -1.2619, -0.8807,  1.6928]],

        [[-0.8783,  0.4861,  0.0658, -0.2935, -1.2247,  1.8444]]],
       grad_fn=<AddBackward0>)
torch.Size([4, 1, 6])



## Decoder functions to display the intermediate outputs and get the final outputs from the decoder


### Self attention outputs from a decoder block

In [85]:
def decoder_block_self_attn_output(x, state_dict, layer_num, tgt_mask = None,need_weights = False):

    # (tgt_len, bsz, embed_dim)
    query_dec = key_dec = value_dec = x

    tgt_len, bsz, embed_dim = x.shape
 
    # (embed_dim*num_heads, embed_dim)
    W_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads

    # (tgt_len, bsz, embed_dim)
    Q_dec,K_dec,V_dec = get_qkv(query_dec, key_dec, value_dec ,W_dec, b_dec)
    
    Q_dec = Q_dec.unsqueeze(0)
    K_dec = K_dec.unsqueeze(0)
    V_dec = V_dec.unsqueeze(0)

    # (1, tgt_len, bsz, embed_dim)
    print(Q_dec.shape, K_dec.shape , V_dec.shape)
    print(tgt_len, bsz * num_heads, head_dim)

    # (1, tgt_len, bsz, embed_dim) -> ( bsz*num_heads, tgt_len , head_dim )
    Q_dec = Q_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec = K_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec = V_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec)
    print()

    src_len = K_dec.size(1)


    attn_mask = tgt_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    

    if need_weights is False:
        attn_output = atten_product_needs_wts_false(Q_dec, V_dec, K_dec, bsz, head_dim, src_len, tgt_len, attn_mask)

        print("Decoder Self Attention = ")
        print(attn_output)
        print()

        op_dec_1 = torch.matmul(attn_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_output.size(1))

        return attn_dec_output, None
    
    else:

        attn_dec_output,attn_wt_matrix_dec = atten_product_needs_wts_true(Q_dec, K_dec, V_dec, bsz, tgt_len, attn_mask)

        print("Decoder Attention output = ")
        print(attn_wt_matrix_dec)
        print()

        op_dec_1 = torch.matmul(attn_dec_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_dec_output.size(1))
    

        return attn_dec_output, attn_wt_matrix_dec


    

In [89]:
need_weights 

False

In [90]:
# need_weights = True

self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(pe_tgt_embeds, state_dict, layer_num = 0, need_weights = need_weights)




torch.Size([4, 1, 6])
torch.Size([18, 6])
torch.Size([1, 4, 1, 6]) torch.Size([1, 4, 1, 6]) torch.Size([1, 4, 1, 6])
4 2 3
Q_dec_0 = 
tensor([[[-1.5232,  0.6053,  0.0973],
         [-1.3771,  0.0798,  0.8412],
         [-2.3907,  0.1429,  0.2497],
         [-2.3907,  0.1429,  0.2497]],

        [[ 0.9852, -1.2133,  0.6450],
         [-0.3394,  0.5405,  0.8201],
         [ 0.1710, -0.7554,  0.9578],
         [ 0.1710, -0.7554,  0.9578]]])

K_dec_0 = 
tensor([[[ 0.2481,  0.0367, -1.0680],
         [ 1.0136, -0.5203, -0.0541],
         [ 1.0379, -0.2288, -1.2677],
         [ 1.0379, -0.2288, -1.2677]],

        [[-0.9174, -0.9733, -0.6989],
         [ 1.2305,  0.1586, -0.1118],
         [ 0.6141, -0.3169, -0.5412],
         [ 0.6141, -0.3169, -0.5412]]])

V_dec_0 = 
tensor([[[-0.2468,  0.7665,  1.3806],
         [ 0.6520,  1.2991,  0.1176],
         [-0.1219,  2.1069,  1.2360],
         [-0.1219,  2.1069,  1.2360]],

        [[ 0.6172,  0.1710, -0.3901],
         [ 0.5168,  1.0014,  0.

In [91]:
self_attn_dec

tensor([[[ 0.4614,  0.5021,  1.5303, -0.4597, -1.4669,  0.7289]],

        [[ 0.4314,  0.4641,  1.3604, -0.4626, -1.4211,  0.7002]],

        [[ 0.4716,  0.4735,  1.4909, -0.3833, -1.4329,  0.6967]],

        [[ 0.4716,  0.4735,  1.4909, -0.3833, -1.4329,  0.6967]]])

In [92]:
def dec_post_self_attn(self_attn_dec, x, state_dict, layer_num):

    # (bsz*tgt_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*tgt_len , embed_dim)
    output_dec_1 = self_attn_dec + x

    # (bsz*tgt_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*tgt_len , embed_dim)
    linear_result_dec_1 = output_dec_1*state_dict["transformer.decoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm1.bias".format(layer_num)]

    layernorm_dec_1 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_1.shape[2:])
    linear_op_dec_1 = layernorm_dec_1(linear_result_dec_1)

    # From torch's implementation
    # print(linear_op_dec_1)

    x = linear_result_dec_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_1.weight
    b = layernorm_dec_1.bias

    linear_result_dec_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_1f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_1 = (linear_result_dec_1f - mean) / (std + epsilon) * w + b


    print("Decoder_{} norm1(x + sa(x))".format(layer_num))
    print(normalized_result_dec_1)
    print()

    # (tgt_len, bsz, embed_dim)
    return normalized_result_dec_1

In [93]:
x_dec = dec_post_self_attn(self_attn_dec, pe_tgt_embeds, state_dict, layer_num = 0)

Decoder_0 norm1(x + sa(x))
tensor([[[-0.2664,  0.8979,  1.5259,  0.1801, -1.2915, -1.0459]],

        [[-0.4785,  0.4683,  0.3127, -0.3071, -1.6347,  1.6393]],

        [[-0.8376,  1.6371, -0.3061, -0.1408, -1.2814,  0.9289]],

        [[-0.8376,  1.6371, -0.3061, -0.1408, -1.2814,  0.9289]]],
       grad_fn=<AddBackward0>)



## Cross attention in decoder 
### query, key, value =  x, mem, mem 

### Cross attention between the encoder's final output and the decoder layer 

In [51]:

memory = output_enc_final
x_dec

tensor([[[-0.2664,  0.8979,  1.5259,  0.1801, -1.2915, -1.0459]],

        [[-0.4785,  0.4683,  0.3127, -0.3071, -1.6347,  1.6393]],

        [[-0.8376,  1.6371, -0.3061, -0.1408, -1.2814,  0.9289]],

        [[-0.8376,  1.6371, -0.3061, -0.1408, -1.2814,  0.9289]]],
       grad_fn=<AddBackward0>)

In [96]:
memory.shape

torch.Size([4, 1, 6])

In [97]:
def decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num, tgt_len, src_len, head_dim, memory_mask = None,need_weights = False):

    # (tgt_len, bsz, embed_dim)
    query_dec_mha = x_dec

    # (src_len, bsz, embed_dim)
    key_dec_mha, value_dec_mha = memory, memory


    tgt_len, bsz, embed_dim = query_dec_mha.shape


    W_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_weight".format(layer_num)]
    b_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_bias".format(layer_num)]


    ########## CHANGE FROM SELF ATTENTION ############

    Q_dec_mha,K_dec_mha,V_dec_mha = get_qkv(query_dec_mha, key_dec_mha, value_dec_mha ,W_dec_mha, b_dec_mha)


    #################################################

    K_dec_mha = K_dec_mha.unsqueeze(0)
    V_dec_mha = V_dec_mha.unsqueeze(0)
    Q_dec_mha = Q_dec_mha.unsqueeze(0)


    Q_dec_mha = Q_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec_mha = K_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec_mha = V_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec_mha)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec_mha)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec_mha)
    print()


    attn_mask = memory_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)



    if need_weights is False:

        attn_output_dec_mha = atten_product_needs_wts_false(Q_dec_mha, V_dec_mha, K_dec_mha, bsz, head_dim, src_len, tgt_len, attn_mask)
        
        print("Cross attention in decoder_{}".format(layer_num))
        print(attn_output_dec_mha)
        print()

        op_dec_mha_1 = torch.matmul(attn_output_dec_mha, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]

        attn_dec_mha_output = op_dec_mha_1.view(tgt_len, bsz, attn_output_dec_mha.size(1))

        return attn_dec_mha_output, None
    
    else: 

    
        attn_dec_mha_output ,attn_wt_matrix_dec_mha = atten_product_needs_wts_true(Q_dec_mha, K_dec_mha, V_dec_mha, bsz, tgt_len, attn_mask)

        print("Decoder mha Attention output = ")
        print(attn_dec_mha_output)
        print()


        op_dec_mha = torch.matmul(attn_dec_mha_output, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]
        attn_dec_output_mha = op_dec_mha.view(tgt_len, bsz, attn_dec_mha_output.size(1))


        return attn_dec_output_mha , attn_wt_matrix_dec_mha

        


In [98]:
# need_weights = True

attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = 0, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = need_weights)




torch.Size([4, 1, 6])
torch.Size([18, 6])
Q_dec_0 = 
tensor([[[-0.8250, -0.5931, -0.4025],
         [-0.0903,  0.5603,  0.7427],
         [ 0.0511, -0.2605, -0.1479],
         [ 0.0511, -0.2605, -0.1479]],

        [[-0.7654, -0.8175,  0.6158],
         [ 0.5966,  0.2556, -0.2416],
         [ 0.7424, -0.3892,  0.5577],
         [ 0.7424, -0.3892,  0.5577]]], grad_fn=<TransposeBackward0>)

K_dec_0 = 
tensor([[[-0.0112,  1.4857, -1.1612],
         [-0.4279,  0.9543, -0.8157],
         [-0.4221,  0.6399, -0.6253],
         [-0.4290,  0.5838, -0.5780]],

        [[ 1.0297, -1.4764, -0.4470],
         [ 1.2456, -1.7020, -0.3399],
         [ 0.5936, -1.1203, -0.1688],
         [ 1.1306, -1.4338, -0.1289]]], grad_fn=<TransposeBackward0>)

V_dec_0 = 
tensor([[[ 0.4800,  0.5421, -0.1278],
         [ 0.1234,  0.9803, -0.7109],
         [ 0.0316,  0.9253,  0.1574],
         [ 0.0551,  0.8494, -0.4480]],

        [[ 0.8969,  0.6116,  0.3663],
         [ 0.9777, -0.2162,  1.2566],
         [ 0.1

In [99]:
attn_dec_mha_output.shape

torch.Size([4, 1, 6])

In [102]:
def decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num):

    # (tgt_len, bsz, embed_dim)
    output_dec_2 = attn_dec_mha_output + x_dec

    linear_result_dec_2 = output_dec_2*state_dict["transformer.decoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation 
    layernorm_dec_2 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_2.shape[2:])
    linear_op_dec_2 = layernorm_dec_2(linear_result_dec_2)

    x = linear_result_dec_2
    w = layernorm_dec_2.weight
    b = layernorm_dec_2.bias

    linear_result_dec_2f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_2f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_2f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_2 = (linear_result_dec_2f - mean) / (std + epsilon) * w + b

    print("norm2(x' + mha(x', mem)), \n where x' = Decoder_curr_layer norm1(x + sa(x))")
    print(normalized_result_dec_2)
    print("\n\n")

    x_dec2_norm = normalized_result_dec_2

    op_dec_1 = torch.matmul(x_dec2_norm, state_dict["transformer.decoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear1.bias".format(layer_num)]
    op_dec_1_relu = torch.nn.functional.relu(op_dec_1)
    op_dec_2 = torch.matmul(op_dec_1_relu, state_dict["transformer.decoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear2.bias".format(layer_num)]

    ff_dec = op_dec_2

    x_dec3_unorm = x_dec2_norm + ff_dec

    linear_result_dec_3 = x_dec3_unorm*state_dict["transformer.decoder.layers.0.norm3.weight"] + state_dict["transformer.decoder.layers.0.norm3.bias"]

    # Layer normalization from Torch's implementation 
    layernorm_dec_3 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_3.shape[2:])
    linear_op_dec_3 = layernorm_dec_3(linear_result_dec_3)

    # From torch's implementation
    x = linear_result_dec_3

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_3.weight
    b = layernorm_dec_3.bias

    linear_result_dec_3f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_3f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_3f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_3 = (linear_result_dec_3f - mean) / (std + epsilon) * w + b

    print("norm3(x'' + ff(x'')) \n where, x'' = Decoder_curr_layer norm2(x' + mha(x'))")
    print(normalized_result_dec_3)
    print()

    return normalized_result_dec_3

def feef_fwd_transformer(dec_output_final, state_dict):

    # (tgt_len, bsz, embed_dim) @ (vocab_size, embed_dim).T -> (tgt_len, bsz, vocab_size)
    final_op = dec_output_final@state_dict["fc.weight"].T + state_dict["fc.bias"]

    return final_op

In [103]:
state_dict["fc.weight"].shape

torch.Size([10, 6])

In [104]:
final_attn_op_decoder = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = 0)

norm2(x' + mha(x', mem)), 
 where x' = Decoder_curr_layer norm1(x + sa(x))
tensor([[[-0.3694,  0.9990,  1.6560, -0.2832, -1.1398, -0.8624]],

        [[-0.6122,  0.8346,  1.0848, -0.6791, -1.5577,  0.9296]],

        [[-0.8573,  1.6522,  0.6363, -0.5546, -1.2864,  0.4097]],

        [[-0.8573,  1.6522,  0.6363, -0.5546, -1.2864,  0.4097]]],
       grad_fn=<AddBackward0>)



norm3(x'' + ff(x'')) 
 where, x'' = Decoder_curr_layer norm2(x' + mha(x'))
tensor([[[-0.0660,  1.2602,  1.3936, -0.4797, -1.1353, -0.9728]],

        [[-0.2781,  1.1315,  0.7439, -0.9239, -1.5632,  0.8898]],

        [[-0.4928,  1.8664,  0.3272, -0.7470, -1.2364,  0.2826]],

        [[-0.4928,  1.8664,  0.3272, -0.7470, -1.2364,  0.2826]]],
       grad_fn=<AddBackward0>)



In [101]:
final_attn_op_decoder.shape

torch.Size([4, 1, 6])

In [56]:
# (tgt_len, bsz, vocab_dim)
final_op = feef_fwd_transformer(final_attn_op_decoder, state_dict)

In [106]:
final_op

tensor([[[ 0.1671,  0.6496,  0.3877, -0.1723, -1.2654,  0.9748, -0.3106,
           0.1239,  0.4715, -1.3378]],

        [[-0.2422,  0.6775,  1.1204, -0.6104, -1.0052,  0.3941,  0.0118,
           0.5823,  0.6551, -1.2290]],

        [[-0.3014,  0.9889,  0.4547, -0.9121, -0.9980,  0.1475, -0.2022,
           0.6603,  1.0222, -1.2369]],

        [[-0.3014,  0.9889,  0.4547, -0.9121, -0.9980,  0.1475, -0.2022,
           0.6603,  1.0222, -1.2369]]], grad_fn=<AddBackward0>)

########################################################################

### Examaple :- 

Transformer with 3 encoders and 3 decoders

In [234]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=4, nhead=2, num_encoder_layers=1, num_decoder_layers=1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        return output

    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 4  # Dimension of the model
num_heads = 2
num_encoder_layers = 3
num_decoder_layers = 3

need_weights = False

src_mask = None
tgt_mask = None
memory_mask = None

max_seq_len = 4


model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model=d_model, nhead=num_heads, num_encoder_layers = num_encoder_layers, num_decoder_layers=num_decoder_layers)
# Source sentence in the source language
src_sentence = torch.tensor([[0], [1], [2], [3]])  # Source sequence

# Target sentence in the target language (translation of the source sentence)
tgt_sentence = torch.tensor([[1], [0], [3], [3]])  # Target sequence

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output)


Source embeddings :- 

tensor([[[-0.8011, -0.1107, -1.6151,  1.4762]],

        [[-0.3233, -1.9852, -0.9699,  0.9073]],

        [[-0.8814, -0.9857,  0.9917, -1.8681]],

        [[ 0.3493, -0.4863,  0.4214, -0.6675]]], grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[-0.2207, -0.4344, -0.1539, -0.3831]],

        [[-1.3731, -0.7337, -0.0944, -0.0221]],

        [[ 0.4218,  0.4912, -1.6193, -2.3457]],

        [[ 0.4218,  0.4912, -1.6193, -2.3457]]], grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-0.8011,  0.8893, -1.6151,  2.4762],
         [ 0.1989,  1.8893, -0.6151,  3.4762],
         [ 1.1989,  2.8893,  0.3849,  4.4762],
         [ 2.1989,  3.8893,  1.3849,  5.4762]],

        [[-0.3233, -0.9852, -0.9699,  1.9073],
         [ 0.6767,  0.0148,  0.0301,  2.9073],
         [ 1.6767,  1.0148,  1.0301,  3.9073],
         [ 2.6767,  2.0148,  2.0301,  4.9073]],

        [[-0.8814,  0.0143,  0.9917, -0.8681],
         [ 0.1186,  1.0143,  1.9917

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [254]:
def get_all_intermediate_outputs(src_sentence, tgt_sentence,d_model, model, num_encoder_layers , num_decoder_layers):

    state_dict = model.state_dict()

    pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence=src_sentence,  tgt_sentence=tgt_sentence, state_dict=state_dict, max_seq_len=max_seq_len, d_model = d_model)
    print("###"*25)
    print("### Encoder Start ###")
    print()

    x_enc = pe_src_embeds

    for lno in range(num_encoder_layers):
        attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(x_enc, state_dict, layer_num = lno, need_weights = False, embed_dim = d_model, num_heads= num_heads, src_mask=None)

        if lno == 0:
            tgt_len, bsz, embed_dim = x_enc.shape

        output_enc_final = encoder_block_post_attn_output(x_enc, attn_enc_output, state_dict, layer_num = lno , bsz = bsz, tgt_len = tgt_len)

        x_enc = output_enc_final

    
    print("### Encoder Done ###")
    print("\n\n\n")
    print("### Decoder Start ###")

    x_dec = pe_tgt_embeds
    memory = x_enc

    for lno in range(num_decoder_layers):

        self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(x_dec, state_dict, layer_num = lno, need_weights = False, tgt_mask=None)
        x_dec = dec_post_self_attn(self_attn_dec, x_dec, state_dict, layer_num = lno)

        attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = lno, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = False, memory_mask=None)
        final_op = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = lno)

        print(pe_tgt_embeds.shape, final_op.shape)
        x_dec = final_op


    final_op = feef_fwd_transformer(final_op, state_dict)

    print("### Decoder Done ###")


    return final_op

    

In [257]:
d_model = 4
num_encoder_layers = 3
num_decoder_layers = 3

final_op = get_all_intermediate_outputs(src_sentence, tgt_sentence, model = model, num_encoder_layers = num_encoder_layers , num_decoder_layers = num_encoder_layers, d_model=d_model)

Source sentence embedding
Word index: 0, Embedding: tensor([-0.8011, -0.1107, -1.6151,  1.4762])
Word index: 1, Embedding: tensor([-0.3233, -1.9852, -0.9699,  0.9073])
Word index: 2, Embedding: tensor([-0.8814, -0.9857,  0.9917, -1.8681])
Word index: 3, Embedding: tensor([ 0.3493, -0.4863,  0.4214, -0.6675])

Target sentence embedding
Word index: 1, Embedding: tensor([-0.2207, -0.4344, -0.1539, -0.3831])
Word index: 0, Embedding: tensor([-1.3731, -0.7337, -0.0944, -0.0221])
Word index: 3, Embedding: tensor([ 0.4218,  0.4912, -1.6193, -2.3457])
Word index: 3, Embedding: tensor([ 0.4218,  0.4912, -1.6193, -2.3457])

PE of src :
tensor([[[0., 1., 0., 1.],
         [1., 2., 1., 2.],
         [2., 3., 2., 3.],
         [3., 4., 3., 4.]]])

PE of tgt :
tensor([[[1., 2., 1., 2.],
         [0., 1., 0., 1.],
         [3., 4., 3., 4.],
         [3., 4., 3., 4.]]])

PE source embeddings : 

tensor([[[-0.8011,  0.8893, -1.6151,  2.4762],
         [ 0.1989,  1.8893, -0.6151,  3.4762],
         [ 1.

In [251]:
output


tensor([[[ 0.3940, -0.5207,  0.5881, -0.0633, -0.3120,  0.5819, -0.4101,
          -1.4635, -0.1699, -0.2797],
         [ 0.5073,  0.1428,  0.3195, -0.0658, -0.9096, -0.1855, -0.1959,
          -1.3236, -0.4518,  0.5941],
         [-0.5948, -0.5942,  0.6483, -0.1120,  0.4457,  0.9227, -0.2608,
          -0.6598,  0.2730, -0.9357],
         [-0.5474, -0.6907,  0.6200, -0.1524,  0.4327,  0.9527, -0.1927,
          -0.5951,  0.2499, -0.9856]],

        [[ 0.4754, -0.8489,  0.4886, -0.2087, -0.3043,  0.6991, -0.1543,
          -1.1643, -0.2179, -0.4919],
         [ 0.3852, -1.0187,  0.3113, -0.3779, -0.3152,  0.6655,  0.2213,
          -0.6228, -0.2607, -0.5969],
         [-0.5107, -0.6982,  0.6293, -0.1455,  0.4188,  0.9588, -0.2144,
          -0.6444,  0.2413, -0.9784],
         [-0.4861, -0.7543,  0.6063, -0.1736,  0.4078,  0.9704, -0.1630,
          -0.5902,  0.2254, -1.0049]],

        [[-0.5983,  1.0723,  0.3436,  0.1102, -0.5498, -0.5710, -0.2827,
          -0.8323, -0.1088,  0.8705

In [252]:
final_op

tensor([[[ 0.3940, -0.5207,  0.5881, -0.0633, -0.3120,  0.5819, -0.4101,
          -1.4635, -0.1699, -0.2797],
         [ 0.5072,  0.1428,  0.3195, -0.0658, -0.9096, -0.1855, -0.1959,
          -1.3236, -0.4518,  0.5941],
         [-0.5948, -0.5942,  0.6483, -0.1120,  0.4457,  0.9227, -0.2608,
          -0.6598,  0.2730, -0.9357],
         [-0.5474, -0.6907,  0.6200, -0.1524,  0.4327,  0.9527, -0.1927,
          -0.5951,  0.2499, -0.9856]],

        [[ 0.4754, -0.8489,  0.4886, -0.2087, -0.3043,  0.6991, -0.1543,
          -1.1643, -0.2179, -0.4919],
         [ 0.3852, -1.0187,  0.3113, -0.3779, -0.3152,  0.6655,  0.2213,
          -0.6228, -0.2607, -0.5969],
         [-0.5107, -0.6982,  0.6293, -0.1455,  0.4188,  0.9588, -0.2144,
          -0.6444,  0.2413, -0.9784],
         [-0.4861, -0.7543,  0.6063, -0.1736,  0.4078,  0.9704, -0.1630,
          -0.5902,  0.2254, -1.0049]],

        [[-0.5984,  1.0723,  0.3436,  0.1102, -0.5498, -0.5710, -0.2827,
          -0.8323, -0.1088,  0.8705